# Building a simple task manager with llmio

This is a demonstration of a Task Manager built with llmio.

The Task Manager is a demo application that allows you to list, create, update, and delete tasks.

Note that since this is only a demonstration, the tasks are stored in memory and will be lost when the application is closed.

### Define the Task model

In [1]:
from dataclasses import dataclass
from typing import Literal


@dataclass
class Task:
    id: int
    name: str
    description: str
    status: Literal["todo", "doing", "done"] = "todo"


# (for demonstration purposes) Define a list of tasks
TASKS: list[Task] = []

### Define the agent

llmio Agents are used to define the behaviour and capabilities of the application.
The Agent requires an instruction that informs the agent of its tasks and desired behaviour.

The Agent class also requires a client to interact with the language model. Here, we use an openai.AsyncOpenAI client.
By default, this will talk to the OpenAI API, but other providers are also supported.

In [2]:
import os

import llmio
from openai import AsyncOpenAI


agent = llmio.Agent(
    instruction="You are a task manager.",
    client=AsyncOpenAI(api_key=os.environ["OPENAI_TOKEN"]),
)

### Define the agent's capabilities with tools

Tools define the capabilities of the agents built with llmio. In this case, we will give the agent the capability to list, create, update, and delete tasks.

In [3]:
# Define a tool to list tasks
@agent.tool
def list_tasks():
    print("** Listing tasks")
    return TASKS


# Define a tool to add a task
@agent.tool
def add_task(name: str, description: str, status: Literal["todo", "doing", "done"] = "todo"):
    print(f"** Adding task '{name}' with status '{status}'")
    task_id = len(TASKS) + 1
    TASKS.append(Task(id=task_id, name=name, description=description, status=status))
    return "Created task with ID " + str(task_id)


# Define a tool to update a task
@agent.tool
def update_task(task_id: int, status: Literal["todo", "doing", "done"] | None = None, description: str | None = None):
    print(f"** Updating task {task_id}")
    for task in TASKS:
        if task.id == task_id:
            task.status = status or task.status
            task.description = description or task.description
            return "Updated task status on task " + str(task_id)
    return "Task not found"


# Define a tool to remove a task
@agent.tool
def remove_task(task_id: int):
    print(f"** Removing task {task_id}")
    for task in TASKS[:]:
        if task.id == task_id:
            TASKS.remove(task)
            return f"OK - removed task {task_id}"
    return "Task not found"

### Interact with the agent

The `Agent.speak` method is used to interact with the agent. This method sends a message to the language model and returns the response.
If tool call requests are received, the agent will execute the appropriate tool method and return the result to the language model, continuing the iteration until only a message is returned.

The `speak` method returns an `llmio.AgentResponse` object containing the received message from the language model, and conversation history so far.
Since the agent is designed to be stateless, in order for parallel execution of messages / commands to be possible, the conversation history is used to maintain the state of the agent. We will therefore store the history and use it to continue the conversation in the later interactions.

In [4]:
# Start a conversation with the agent. This initializes a message history that we can use to keep track of the conversation.
response = await agent.speak("Hi!")

for message in response.messages:
    print(f"** Bot: {message}")

** Bot: Hello! How can I assist you today?


In [5]:
response = await agent.speak("List my tasks", history=response.history)

for message in response.messages:
    print(f"** Bot: {message}")

TASKS

** Listing tasks
** Bot: It seems you don't have any tasks currently. Would you like to add a new task?


[]

### Add a message handler

Instead of waiting for messages to return, we can add a message handler to process agent messages as they come in.

This can be used to call a webhook or to process the message in some other way.

In this example, we will use a message handler to print the agent's response.

In [6]:
@agent.on_message
async def handle_message(message: str) -> None:
    print(f"** Bot (message_handler): '{message}'")

### Continue the interaction, but using the message handler to print the messages

In [7]:
response = await agent.speak("Add a grocery task with a description to buy eggs, bacon, bread, tomatoes and orange juice", history=response.history)

TASKS

** Adding task 'Grocery' with status 'todo'
** Bot (message_handler): 'I've added a grocery task to buy eggs, bacon, bread, tomatoes, and orange juice. If you need anything else, feel free to let me know!'


[Task(id=1, name='Grocery', description='Buy eggs, bacon, bread, tomatoes and orange juice', status='todo')]

In [8]:
response = await agent.speak("Also add a task to clean the house", history=response.history)

TASKS

** Adding task 'Clean the house' with status 'todo'
** Bot (message_handler): 'I've added the task to clean the house. If there's anything more you'd like to do, just let me know!'


[Task(id=1, name='Grocery', description='Buy eggs, bacon, bread, tomatoes and orange juice', status='todo'),
 Task(id=2, name='Clean the house', description='Thoroughly clean all rooms including dusting, vacuuming, and mopping.', status='todo')]

In [9]:
response = await agent.speak("i also need to buy some milk, can you add that to the grocery list?", history=response.history)

TASKS

** Updating task 1
** Bot (message_handler): 'I've updated the grocery task to include milk. If you need any further assistance, just let me know!'


[Task(id=1, name='Grocery', description='Buy eggs, bacon, bread, tomatoes, orange juice, and milk', status='todo'),
 Task(id=2, name='Clean the house', description='Thoroughly clean all rooms including dusting, vacuuming, and mopping.', status='todo')]

In [10]:
response = await agent.speak("The grocery task is done", history=response.history)

TASKS

** Updating task 1
** Bot (message_handler): 'I've marked the grocery task as done. If there's anything else you'd like to manage or update, feel free to ask!'


[Task(id=1, name='Grocery', description='Buy eggs, bacon, bread, tomatoes, orange juice, and milk', status='done'),
 Task(id=2, name='Clean the house', description='Thoroughly clean all rooms including dusting, vacuuming, and mopping.', status='todo')]

In [11]:
response = await agent.speak("remove all completed tasks", history=response.history)

TASKS

** Removing task 1
** Bot (message_handler): 'I've removed the completed grocery task. If you need help with anything else, just let me know!'


[Task(id=2, name='Clean the house', description='Thoroughly clean all rooms including dusting, vacuuming, and mopping.', status='todo')]

In [12]:
response = await agent.speak("can you list my current tasks?", history=response.history)

TASKS

** Listing tasks
** Bot (message_handler): 'You currently have one task:

1. **Clean the house**
   - Description: Thoroughly clean all rooms including dusting, vacuuming, and mopping.
   - Status: To Do

If you need to update or add anything, just let me know!'


[Task(id=2, name='Clean the house', description='Thoroughly clean all rooms including dusting, vacuuming, and mopping.', status='todo')]